<a href="https://colab.research.google.com/github/mqjasper/EDTE4330AT1-1/blob/main/EDTE4330_AT1_Chem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Soaps, detergents, and polar molecules in Organic Chemistry

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1dcR8WHEtSOzaluLEjELKwtUrT45fY_k9/view?usp=sharing)

This notebook is based on the excellent [Kaggle tutorial](https://www.kaggle.com/vladislavkisin/tutorial-ml-in-chemistry-research-rdkit-mol2vec) from [Vlad Kisin](https://www.kaggle.com/vladislavkisin). In this example, you'll learn how to read a Chemistry datafile and create predictive models of [lipophilicity](https://en.wikipedia.org/wiki/Partition_coefficient#Partition_coefficient_and_log_P).

Lipophilicity is the ability of a chemical compound to dissolve in non-polar (fatty or oily) solvents. In simple terms, if you had a glass of oil and water (which will separate with one on top of the other as in the figure below), then lipophilicity is the proportion of how much a chemical dissovles in the water portion versus the oil portion. In the figure below there are 3 molecules in water to every 1 molecule in oil. P is 3 and the log P is $\log_{10}{3} = 0.477$.

Lipophilicity contributes to the absorption, distribution, metabolism, excretion, and toxicity of a pharmaceutical and contributes to a drug's potency and selectivity.

![Figure1](https://github.com/tonyreina/chemistry/blob/main/logP.png?raw=1)

I'll demonstrate how to load the raw data from a CSV file and use the [RD-Kit](https://github.com/rdkit/rdkit) and [Mol2Vec](https://pubs.acs.org/doi/10.1021/acs.jcim.7b00616) packages to create features based on the chemical formula of a molecule.

# Install & import necessary libraries

In [ ]:
%pip install seaborn
%pip install pandas
%pip install matplotlib
%pip install git+https://github.com/samoturk/mol2vec
!wget https://raw.githubusercontent.com/tonyreina/mol2vec/master/mol2vec/features.py -O /usr/local/lib/python3.10/dist-packages/mol2vec/features.py
%pip install scikit-learn
%pip install py3Dmol
%pip install rdkit-pypi

## Import installed libraries
Now that we have all the necessary libraries installed we need to tell python to use them

In [ ]:
%matplotlib inline

import numpy as np                #!pip install numpy
import pandas as pd               #!pip install pandas
import matplotlib.pyplot as plt   #!pip install matplotlib
import seaborn as sns             #!pip install seaborn

## Load SMILES structures from a file

I have uploaded a small file with a selection of relevant molecules that will be used at some point during the lab. This will be imported to the session and loaded for later use. Below is an explanation of how the data is provided using a molecular structuring system known as SMILES:

[SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) stands for the **S**implified **M**olecular **I**nput **L**ine **E**ntry **S**ystem which is a method for describing the structure of chemical species using short ASCII strings. Essentially it's a short-hand notation for describing a molecule including information about which atoms are connected together, the types of chemical bonds, and the 2D structure. In SMILES notation, 'C' represents a carbon atom, 'N' represents a nitrogen atom, 'Cl' represents a Chlorine atom, and '=' indicates a double bond between the adjacent atoms. For example, in the figure below the SMILES notation for the top/left molecule is **Cl
ClC\[C@H\](\[C@@H\](C)Cl)Cl**.

A good resource for learning about SMILES notation is:

Quirós, M., Gražulis, S., Girdzijauskaitė, S. et al. Using SMILES strings for the description of chemical connectivity in the Crystallography Open Database. J Cheminform 10, 23 (2018). https://doi.org/10.1186/s13321-018-0279-6



---



#**IMPORTANT: INCLUDE CSV UPLOAD SPACE FOR STUDENTS TO CREATE THEIR OWN TABLE RATHER THAN USE PROVIDED STRUCTURES**

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/mqjasper/EDTE4330AT1-1/main/structures.csv', names=['SMILES', 'Name']) #load the CSV file
df.head()  # Display a few rows

## What is the goal for this lab?

We'd like to take **SMILES** structures, interpret it in 2D and 3D space, manipulate and analyse these structures.

## RD-Kit
[RD-Kit](https://github.com/rdkit/rdkit) is an open-source (BSD-3) Python package for data scientists to work with Chemistry data. It has some excellent modules that allow us to preprocess data from chemistry experiments. We installed this library in the first code block.

With RD-Kit we can transform the SMILES notation into a more useful Python object called a *Mol* (for molecule). Mol objects contain the atoms, bonds, connectivity and coordinates of a molecule. It adds important 3D information, such as the distance between atoms, that may be useful to our model. We can use *Mol* objects to craft features of the chemical to provide as inputs to our model.

In [ ]:
#Importing Chem module
from rdkit import Chem

#Method transforms smiles strings to mol rdkit object
df['mol'] = df['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))

df.head()

## Plot some chemical structures

Once we have our molecules as *Mol* Python objects, then RD-Kit allows us to plot the chemical structures easily.

In [ ]:
from rdkit.Chem import Draw

number_to_print = 20
mols = df['mol'][:number_to_print]

# AddHs function adds H atoms to a MOL (as Hs in SMILES are usualy ignored)
df['mol'] = df['mol'].apply(lambda x: Chem.AddHs(x))

#MolsToGridImage allows to show several molecules in a grid
Draw.MolsToGridImage(mols, molsPerRow=5, useSVG=True, legends=list(df['SMILES'][:number_to_print].values))

## View it in 3D
There's a nice Python package called [py3Dmol](https://3dmol.csb.pitt.edu/) which creates 3D images from the *Mol* object. This is not necessary, but it does create pretty pictures of our molecules that can be rotated and zoomed.

In [ ]:
import py3Dmol  # pip install py3Dmol
from ipywidgets import interact,fixed,IntSlider
import ipywidgets

def show3D_molecule(idx, style):
    """
    Show molecule in 3D
    """
    mblock = Chem.MolToMolBlock(df['mol'].iloc[idx])
    viewer = py3Dmol.view(width=300, height=300)
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    viewer.rotate(45, "y", animationDuration=1)

    viewer.zoomTo()

    print(f"SMILES notation: {df['SMILES'].iloc[idx]}\nRotate me!");

    return viewer.show()

interact(show3D_molecule,
         idx=ipywidgets.IntSlider(min=0,max=len(df["mol"])-1,
                                  step=1, value=3064,
                                  description="Molecule"),
         style=ipywidgets.Dropdown(options=['line', 'stick', 'sphere'],
                                   value='stick',
                                   description='Style:'));

## Use RD-Kit to create features for the model

Now that we have the molecule in *Mol* format, we can use RD-Kit's built in functions to create new features for our model. For example, we can add columns for the number of total atoms in the molecule and the number **Chloride (Cl)** atoms in the molecule. These features might be important in predicting **logP**.

Note: The first function we use is `Chem.AddHs`. Typically, the SMILES format does not explicitly show the **Hydrogen atoms (H)**. They are implied. It actually helps us a bit in our model accuracy to specifically add these back. (Try commenting out the next line to see its effect on model performance.)

In [ ]:
# AddHs function adds H atoms to a MOL (as Hs in SMILES are usualy ignored)
df['mol'] = df['mol'].apply(lambda x: Chem.AddHs(x))

## Additional features

Let's add some other features such as the number of atoms in the molecule and the number of "heavy" atoms in the molecule. These might be important in determining the logP value. Look through the [RD-Kit documentation](https://www.rdkit.org/docs/source/rdkit.Chem.Lipinski.html) to add more features if you'd like to get a better model.

In [ ]:
# GetNumAtoms() method returns a general nubmer of all atoms in a molecule
df['num_of_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms())

# GetNumHeavyAtoms() method returns a nubmer of all atoms in a molecule with molecular weight > 1
df['num_of_heavy_atoms'] = df['mol'].apply(lambda x: x.GetNumHeavyAtoms())

In [ ]:
#We're going to searches patterns and use it for a list of most common atoms only
def number_of_atoms(atom_list, df):
    for i in atom_list:
        df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

number_of_atoms(['C', 'O', 'N', 'Cl', 'P', 'Br', 'F'], df)

In [ ]:
from rdkit.Chem import Descriptors
df['tpsa'] = df['mol'].apply(lambda x: Descriptors.TPSA(x)) #https://en.wikipedia.org/wiki/Polar_surface_area
df['mol_w'] = df['mol'].apply(lambda x: Descriptors.ExactMolWt(x)) # https://en.wikipedia.org/wiki/Molecular_mass
df['num_valence_electrons'] = df['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x)) # https://en.wikipedia.org/wiki/Valence_electron
df['num_heteroatoms'] = df['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))

In [ ]:
df.head()